# INTRO

The goal of this notebook is to create a database that can be accessed by users with different privileges. The final output of this notebook is illustrated in the diagram below and the following steps:

* 1 - Create the database
* 2 - Create the schema
* 3 - Create the table
    * 3.1 - Explore the different datatypes in the csv file
    * 3.2 - Create a table with the most efficient datatype for each column
    * 3.3 - Fill the boston_crimes table using data stored in a csv file (boston.csv)
* 4 - Create user groups
* 5 - Create users for each user group
* 6 - Summary



![picture](https://dq-content.s3.amazonaws.com/250/goal.png)

Let us start!!! 


![picture](https://i.imgflip.com/2xlv1w.jpg)

## 1. Create the database

Let' start by creating a database and a scheme. If we use an IKEA analogy, the database is the part of the store where all the products are store. The schema is the map that tells you where the different ailes are. So you know which aile to go to to find your product.

In [2]:
import psycopg2 #to connect to postgres
import getpass #to prompt user to insert password in a secure manner
import csv #to read csv files
import numpy as np
import pandas as pd
from prettytable import PrettyTable #to print tables in a neat pretty format

Creating the database 'crime_db'

In [5]:
#indicate the parameters to connect to postgres
host = 'localhost'
port = '5432'
database = 'postgres' # this is the default database to connect to as we do not have any existing database yet
username = 'postgres'
#for security purposes, ask python to prompt you for the password
password = getpass.getpass(prompt = 'Password:')

#let's connect to the default database
conn = psycopg2.connect(dbname = database,user = username, password = password)
#Creating a databse cannot be part of a transaction block. This is because it requires access to the system catalogs which are outside the scope of a transaction. 
conn.autocommit = True
cur = conn.cursor()
new_database_name = 'crime_db'
#let us create the new database. If already created, we will print an appropriate message to convey that. 
try:
    cur.execute(f'CREATE DATABASE {new_database_name};')
except:
    print(f'{new_database_name} database is already created')

conn.autocommit = False
conn.close()






crime_db database is already created


## 2. Create the Schema

Creating the schema crimes:

In [7]:
database = 'crime_db'
schema_crimes = 'crimes'
conn = psycopg2.connect(dbname = database, user = 'postgres', password = password)
cur = conn.cursor()
try:
    cur.execute(f'CREATE SCHEMA {schema_crimes};')
except:
    print('Schema already exists')

Schema already exists


We will keep the connection open till the end of the project.

By creating our database and schema, we now have this infrastructure:

![picture](https://dq-content.s3.amazonaws.com/250/create_db.png)

## 3. Create the table

### 3.1 - Explore the different datatypes in the csv file


Now let us starting filling our schema with tables.
First, we need to read our data to understand what data types we have. 

In [8]:
with open('boston.csv','r') as file:
    #use first row as column headers
    col_headers = next(file)
    #split the string into a list of column names
    col_headers = col_headers.split(',')
    #remove the '\n' from the last element
    col_headers[-1] = col_headers[-1].strip('\n')
    #use next row after first as the first row with values
    first_row = next(file)
    print(f'The file has the following columns: {col_headers} \n')
    print(f'The first row in the file has the following values: {first_row}')

The file has the following columns: ['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'] 

The first row in the file has the following values: 1,619,LARCENY ALL OTHERS,2018-09-02,Sunday,42.35779134,-71.13937053



Now we need to know how many unique values does each column take. This will help us in choosing the data type for each feature. I will do that by creating a dictionary where each key is the name of the column and the values correspond to the unique values that the this column/feature has. 

In [9]:
data_values = {}
days_of_the_week = ['Monday', 'Tuesday', 'Wednesday', 'Thursday','Friday','Saturday','Sunday'] 
with open('boston.csv','r') as file:
    #skip the header
    next(file)
    #read the file into rows 
    rows = csv.reader(file)
    i=0
    for row in rows:
        i=i+1
        for column in col_headers:
            #for each column find the row index that has its value
            column_index = col_headers.index(column)
            if column in data_values:
                #use the column index to add the corresponding row value to the dictionary (using add to make sure only unique values are saved)
                data_values[column].add(row[column_index])
                
                
            else:
                data_values[column] = set([row[column_index]])


Let us check the number of unique values that each column/feature has. This will inform our decision of the most approprate data type to use when saving it in our database. 

In [10]:
data_values.keys()

dict_keys(['incident_number', 'offense_code', 'description', 'date', 'day_of_the_week', 'lat', 'long'])

In [11]:
for key, values in data_values.items():
    print(f'{key} has {len(data_values[key])} unique values. One example is ------> {next(iter(values))}\n')

incident_number has 298329 unique values. One example is ------> 149914

offense_code has 219 unique values. One example is ------> 3501

description has 239 unique values. One example is ------> M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY

date has 1177 unique values. One example is ------> 2017-09-08

day_of_the_week has 7 unique values. One example is ------> Sunday

lat has 18177 unique values. One example is ------> 42.29817352

long has 18177 unique values. One example is ------> -71.06175129



The values of all the columns already give us an idea of what the most memory-efficient data type is to represent them. Let us the range for each data type.

In [12]:
def find_range(column):
    ''' Finds the range [min,max] and data type for each column
    Arguments:
    ==========
    * column: a set taking from a dictionary
    Returns:
    ========
     * min: min value in the column set. The type depends on the datatype of values in column (int, float). For strings, it will return the size of shortest string
     * min: max value in the column set. The type depends on the datatype of values in column (int, float). For strings, it will return the size of longest string
     * data_type: string containing the datatype of the column
     '''
    #set the min/max to dummy values
    max_value = -np.Inf
    min_value = np.Inf
    #iterate over each item of the column
    for item in column:
        #try if the item type is an integer
        try:
            item = int(item)
            if item > max_value:
                    max_value = item
            if item < min_value:
                min_value = item
        #if that gives an error, then it is a string or a date
        except:
            try:
                item = float(item)
                if item > max_value:
                        max_value = item
                if item < min_value:
                    min_value = item
            except:
                if len(item) > max_value:
                    max_value = len(item)
                if len(item) < min_value:
                    min_value = len(item)

    data_type = type(item)              
    return data_type, min_value, max_value


In [13]:
# for key, values in data_values.items():
#     print(f'{key} has {len(data_values[key])} unique values. One example is ------> {next(iter(values))}\n')
for key, values in data_values.items():
    data_type, min_value, max_value = find_range(values)
    print(f'{key} has {len(data_values[key]):,} unique values, is of datatype {data_type} and ranges between ({min_value},{max_value}). One example is ------> {next(iter(values))}\n')


incident_number has 298,329 unique values, is of datatype <class 'int'> and ranges between (1,298329). One example is ------> 149914

offense_code has 219 unique values, is of datatype <class 'int'> and ranges between (111,3831). One example is ------> 3501

description has 239 unique values, is of datatype <class 'str'> and ranges between (5,58). One example is ------> M/V ACCIDENT INVOLVING PEDESTRIAN - INJURY

date has 1,177 unique values, is of datatype <class 'str'> and ranges between (10,10). One example is ------> 2017-09-08

day_of_the_week has 7 unique values, is of datatype <class 'str'> and ranges between (6,9). One example is ------> Sunday

lat has 18,177 unique values, is of datatype <class 'float'> and ranges between (42.2324133,42.39504158). One example is ------> 42.29817352

long has 18,177 unique values, is of datatype <class 'float'> and ranges between (-71.17867378,-70.96367615). One example is ------> -71.06175129



Given the information we now have, let us associate a datatype to each column:
* **incident number** -> **INT** (should be good as long as we have less than 2.1 billion incidents)

* **offense_code** -> **SMALLINT** (as long as we have less than 32,000 offense_codes. At present, offense codes are between 111-3,831)

* **description** -> **VARCHAR(120)** (the maximum description has 58 characeters let us round that to 60 and double it to be on the safe side)

* **date** -> DATE

* **day_of_the_week** -> **VARCHAR(9)** (wednesday has 9 letters). However, I will replace that with ENUM datatype. It will transform the datatype from string to integer.

* **lat** -> **DECIMAL(10,8)** 10 digits in total with 8 after the decimal point

* **long** -> **DECIMAL(10,8)** 10 digits in total with 8 after the decimal point

Let us create some enumerated datatypes to optimise memory use. I will choose day_of_the_week. offense_code and description can be enumerated as well but they would require some python to iterate over all unique values. Otherwise, logging them manually will be very tedious and is not the objective of this notebook.

In [19]:
conn.commit()

### 3.2 - Create a table with the most efficient datatype for each column



In [21]:
sql_command =f"CREATE TYPE enum_day_of_the_week AS ENUM {data_values['day_of_the_week']};"
sql_command = sql_command.replace('{','(').replace('}',')')
try:
    cur.execute(sql_command)
except:
    print('ENUM dataype already exists')

ENUM dataype already exists


Now let us create our table in the crimes schema

In [23]:
conn.commit()

In [24]:
try:
    cur.execute('''
                CREATE TABLE crimes.boston_crimes (
                    incident_number INT PRIMARY KEY,
                    offense_code SMALLINT,
                    description VARCHAR(120),
                    date DATE,
                    day_of_the_week enum_day_of_the_week,
                    lat DECIMAL(10,8),
                    long DECIMAL(10,8)
                );
    ''')
except:
    print('Table already exists')
conn.commit()

Table already exists


Let us check our table and the datatypes of each column

In [25]:
query = ''' 
SELECT column_name, data_type
FROM information_schema.columns
WHERE table_schema = 'crimes'
  AND table_name = 'boston_crimes';'''

results = cur.execute(query)
results = cur.fetchall()
for row in results:
    print(row)

('incident_number', 'integer')
('offense_code', 'smallint')
('description', 'character varying')
('date', 'date')
('day_of_the_week', 'USER-DEFINED')
('lat', 'numeric')
('long', 'numeric')


All looking good! Now let us use the csv file to fill the table.

### 3.3 - Fill the boston_crimes table using data stored in a csv file (boston.csv)


In [27]:
try:
    with open('boston.csv','r') as file:
        cur.copy_expert("COPY crimes.boston_crimes FROM STDIN WITH CSV HEADER;", file)
except:
    print('File already loaded')

File already loaded


Let us check that the table was successfully loaded

In [29]:
conn.commit()
query = ''' SELECT * FROM crimes.boston_crimes LIMIT 10'''

cur.execute(query)

rows = cur.fetchall()

table = PrettyTable(col_headers)
for row in rows:
    table.add_row(row)

print(table)

+-----------------+--------------+--------------------------------------------+------------+-----------------+-------------+--------------+
| incident_number | offense_code |                description                 |    date    | day_of_the_week |     lat     |     long     |
+-----------------+--------------+--------------------------------------------+------------+-----------------+-------------+--------------+
|        1        |     619      |             LARCENY ALL OTHERS             | 2018-09-02 |      Sunday     | 42.35779134 | -71.13937053 |
|        2        |     1402     |                 VANDALISM                  | 2018-08-21 |     Tuesday     | 42.30682138 | -71.06030035 |
|        3        |     3410     |            TOWED MOTOR VEHICLE             | 2018-09-03 |      Monday     | 42.34658879 | -71.07242943 |
|        4        |     3114     |            INVESTIGATE PROPERTY            | 2018-09-03 |      Monday     | 42.33418175 | -71.07866441 |
|        5        | 

In [ ]:
conn.commit()

Now doesn't that look pretty!!!!

![picture](https://media.tenor.com/Osi-RwGWBwIAAAAM/pretty-i-feel-pretty.gif)

We have now added our first table into our crimes schema. Here is an updated diagram of my progress:

![picture](https://dq-content.s3.amazonaws.com/250/table_created.png)

Now it is time to create user groups.

## 4 - Create user groups


I want to add two user groups: readonly and readwrite. First, let me make sure that there are no privileges inherited from the public group and on the public schema. 

In [43]:
#revoke all privileges of the public group on the public schema
cur.execute('REVOKE ALL ON SCHEMA public FROM public')
#revoke all privileges of public on the crime_db database
cur.execute('REVOKE ALL ON DATABASE crime_db FROM public')

No I am sure no user group will inadvertently inherit privileges from the public group, I can create two new user groups.

In [44]:
conn.commit()

In [45]:
#create a group with NOLOGIN option. We want to make sure groups are not used to login. Login should be only done by providing a user
#create readonly group
cur.execute('CREATE GROUP readonly NOLOGIN;')
#create readwrite group
cur.execute('CREATE GROUP readwrite NOLOGIN;')

Now let us select the privileges suitable for each group:
* The readonly group is supposed to only have privileges to perform SELECT queries
* The readwrite group to be able to perform SELECT, INSERT, DELETE, and UPDATE queries

In [46]:
conn.commit()

In [47]:
#grant SELECT privileges to readyonly group
cur.execute('GRANT SELECT ON ALL TABLES IN SCHEMA crimes TO readonly')
#grant SELECT, INSERT, DELETE and UPDATE privileges to readwrite group
cur.execute('GRANT SELECT, INSERT, DELETE, UPDATE ON ALL TABLES IN SCHEMA crimes TO readwrite')

Now let us give both groups the privilege to connect and use all tables in the crimes_db database and the crimes schema

In [48]:
conn.commit()

In [49]:
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readonly')
cur.execute('GRANT CONNECT ON DATABASE crime_db TO readwrite')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readonly')
cur.execute('GRANT USAGE ON SCHEMA crimes TO readwrite')

Now let us create new users for each user group!

![picture](https://i.swncdn.com/media/800w/via/images/2022/07/01/26215-minions-universal-2_source_file.jpg)

## 5 - Create users for each user group

In [50]:
conn.commit()
#create a data_analyst with password 'secret1'
cur.execute("CREATE USER data_analyst WITH PASSWORD 'secret1'")
#assign the user to the readonly group
cur.execute("GRANT readonly TO data_analyst")
#create a data_scientist with password 'secret2'
cur.execute("CREATE USER data_scientist WITH PASSWORD 'secret2'")
#assign the user to readwrite group
cur.execute("GRANT readwrite TO data_scientist")



Let us do some tests to make sure everything as we expect it to be: 
* Does the readwrite user group has the correct privileges?

In [ ]:
cur.execute('''SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readwrite'; ''')
cur.fetchall()

* Does the readonly user group has the correct privileges?

In [54]:
cur.execute('''SELECT grantee, privilege_type
    FROM information_schema.table_privileges
    WHERE grantee = 'readonly'; ''')
cur.fetchall()

[('readonly', 'SELECT')]

In [ ]:
conn.commit()
conn.close()

## 6 - Summary

We have achieved the objective of this notebook: 
* Created a crime_db database
* Create a crimes schema
* Created a boston_crimes table in crimes schema (read from a csv file)
* Created two user groups with different privileges
* Created a user in each user group 


![picture](https://dq-content.s3.amazonaws.com/250/goal.png)